In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
import re, datetime
from lifelines import KaplanMeierFitter
from datetime import date, timedelta

In [32]:
dict1={
"4.3" : "26-6-2013 20:00:00" ,
"4.4" :  "25-6-2014 12:15:00" ,
"4.5" : "24-6-2015 20:00:00" ,
"4.6" :"22-6-2016 11:00:00",
"4.7" :  "28-6-2017 09:50:00",




}

In [ ]:
arr=[
"4.4" ,
"4.5" ,
"4.6"  ,
"4.7",
"4.8" 
]

In [33]:
def keepVal(activity):
    if activity=='NEW'or activity=='Assignee':
        return 1
    return 0

In [34]:
dX=pd.read_csv('db\\EC2.csv')


In [35]:
#read the CSV file
df=pd.read_csv('db\\EC2.csv')

df['creation_ts']=pd.to_datetime(df['creation_ts'])

In [36]:
df = df[df['creation_ts']>='01-01-2013 00:00:00']

In [37]:
allD=df

In [38]:
len(df.bug_id.unique())

20803

In [39]:
df['keep']=df.apply(lambda x : keepVal(x['activity']), axis=1)
df = df[df['keep']==1]
df = df[['When', 'Who', 'activity', 'bug_id', 'bug_severity', 'bug_status',
       'creation_ts', 'resolution', 'version']]

In [40]:
df1=df
df2=df

In [41]:
df2=df2.groupby(by='bug_id', as_index=False).agg({'activity': pd.Series.nunique})
df2 = df2[df2['activity']!=2]

In [42]:
dfList = df2['bug_id'].tolist()

In [43]:
gap =allD[~allD.bug_id.isin(dfList)]

In [44]:
df=gap

In [45]:
df['When']=pd.to_datetime(df['When'])
df['creation_ts']=pd.to_datetime(df['creation_ts'])

C:\Users\zeinab\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\zeinab\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [46]:
xData=pd.DataFrame()
id_before_release=0
id_after_release=0
id_RBRB=0
id_RBFB=0
id_RBRA=0
id_RBFA=0
id_RARA=0
id_RAFA=0
for x in dict1:
    data=pd.DataFrame()
    
    start_date = dict1[x]
    print(start_date)
    date_time_obj = datetime.datetime.strptime(start_date, '%d-%m-%Y %H:%M:%S')
    print('st',date_time_obj)
    k=date_time_obj + timedelta(60)
    version= x
   # print(x,dict[x])
    #mask = (df['creation_ts'] > start_date) & (df['creation_ts'] <= end_date) & (("4.4.1"==df['version'])  | ("4.4"==df['version']))
    #mask = (df['creation_ts'] > start_date) & (df['creation_ts'] <= end_date) & ("4.4"==df['version'])
    mask= (version==df['version'])  
    #the bug before release 
   # maskp = (version==df['version'])  & (df['creation_ts'] < start_date)
    #the bug after release 
    maskp = (version==df['version'])  & (df['creation_ts'] >= start_date)  & (df['creation_ts'] <= k)
    
    maskpp = (version!=df['version'])  & (df['creation_ts'] >= start_date)  & (df['creation_ts'] <= k)


    #the bug created and resolved & before release 
    mask1 = (df['When'] < start_date) & (df['creation_ts'] <= start_date) & (version==df['version']) & ("RESOLVED"==df['activity']) 
    #the bug created and fixed before release 
    mask2 = (df['When'] < start_date) & (df['creation_ts'] <= start_date) & (version==df['version']) & ("RESOLVED"==df['activity'])  & ("FIXED"==df['resolution'])

    #the bug created before release and resolved  after release 
    mask3 = (df['When'] > start_date) & (df['creation_ts'] <= start_date) & (version==df['version']) & ("RESOLVED"==df['activity'])
    #the bug created before release and fixed after release 
    mask4 = (df['When'] > start_date) & (df['creation_ts'] <= start_date) & (version==df['version']) & ("RESOLVED"==df['activity']) & ("FIXED"==df['resolution']) 
    #the bug created after release  and resolved
    mask5 = (df['creation_ts'] >= start_date) & (version==df['version']) & ("RESOLVED"==df['activity']) 
    #the bug created after release  and fixed 
    mask6 = (df['creation_ts'] >= start_date) & (version==df['version']) & ("RESOLVED"==df['activity']) & ("FIXED"==df['resolution'])


    dff = df.loc[mask]
    dffp = df.loc[maskp]
    dffpp = df.loc[maskpp]
    dff1 = df.loc[mask1]
    dff2 = df.loc[mask2]
    dff3 = df.loc[mask3]
    dff4 = df.loc[mask4]
    dff5 = df.loc[mask5]
    dff6 = df.loc[mask6]
    id_RF=dffp.bug_id.unique()
    id_RS=dffpp.bug_id.unique()

    
    TOTAL=len(dff.bug_id.unique())
    data=pd.DataFrame({"version": [version] , "total":[ TOTAL] , 'id_RV':[id_RF]
   , 'id_RDV':[id_RS]
  
            
   })
    

    xData=xData.append(data)

26-6-2013 20:00:00
st 2013-06-26 20:00:00
25-6-2014 12:15:00
st 2014-06-25 12:15:00
24-6-2015 20:00:00
st 2015-06-24 20:00:00
22-6-2016 11:00:00
st 2016-06-22 11:00:00
28-6-2017 09:50:00
st 2017-06-28 09:50:00


In [47]:
xData

,id_RDV,id_RV,total,version
0,"[413906, 415268, 413492, 415023, 414122, 41544...","[412809, 413493, 414108, 412154, 413213, 41413...",1841,4.3
0,"[441990, 441507, 442152, 441697, 441217, 44142...","[440763, 439027, 442285, 441821, 440300, 44001...",3133,4.4
0,"[473986, 474943, 473432, 473850, 473921, 47352...","[473503, 474002, 471352, 472426, 472413, 47120...",1863,4.5
0,"[496868, 496969, 496970, 497013, 496756, 49678...","[496686, 496661, 496769, 496813, 496886, 49686...",2235,4.6
0,"[518956, 518895, 518985, 519055, 519365, 51915...","[519026, 519032, 519056, 518905, 519152, 51923...",2263,4.7


In [48]:
for index, row in xData.iterrows():
    df3=df[df.bug_id.isin(row['id_RV'])]
   # print(len(row['id_RAFA']))
    version=row['version']
    df3['keep']=df3.apply(lambda x : keepVal(x['activity']), axis=1)
    df3 = df3[df3['keep']==1]
    df3 = df3[['When', 'activity', 'bug_id']]

    df3['When']=pd.to_datetime(df3['When'])
    #print (df3.groupby('bug_id').groups)
    df3=df3.groupby(['bug_id','activity'])['When'].min().reset_index()
    df3['diff'] = df3.groupby('bug_id')['When'].transform(lambda x: (x.max()-x.min()).days)
    df3=df3.groupby('bug_id').first().reset_index()
    df3= df3[['bug_id',
     'When',
     'diff']]
    df3['done'] = True
    df3.to_csv('db\\aEclipse-RV_'+str(version)+'.csv',index=False)
    
    
    
    df4=df[df.bug_id.isin(row['id_RDV'])]
   # print(len(row['id_RAFA']))
    df4['keep']=df4.apply(lambda x : keepVal(x['activity']), axis=1)
    df4 = df4[df4['keep']==1]
    df4 = df4[['When', 'activity', 'bug_id']]

    df4['When']=pd.to_datetime(df4['When'])
    #print (df3.groupby('bug_id').groups)
    df4=df4.groupby(['bug_id','activity'])['When'].min().reset_index()
    df4['diff'] = df4.groupby('bug_id')['When'].transform(lambda x: (x.max()-x.min()).days)
    df4=df4.groupby('bug_id').first().reset_index()
    df4= df4[['bug_id',
     'When',
     'diff']]
    df4['done'] = False
    df4.to_csv('db\\aEclipse-RDV_'+str(version)+'.csv',index=False)
    
   # print (row['id_RAFA'], row['id_RARA'])
    #break

#fg=tData[0]

C:\Users\zeinab\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\zeinab\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [49]:
kmf = KaplanMeierFitter()

draft1 = pd.read_csv('db\\aEclipse-RV_4.3.csv')
draft2 = pd.read_csv('db\\aEclipse-RDV_4.3.csv')
data1=['4.3',draft1,draft2]
draft3 = pd.read_csv('db\\aEclipse-RV_4.4.csv')
draft4 = pd.read_csv('db\\aEclipse-RDV_4.4.csv')
data2=['4.4',draft3,draft4]
draft5 = pd.read_csv('db\\aEclipse-RV_4.5.csv')
draft6 = pd.read_csv('db\\aEclipse-RDV_4.5.csv')
data3=['4.5',draft5,draft6]
draft7 = pd.read_csv('db\\aEclipse-RV_4.6.csv')
draft8 = pd.read_csv('db\\aEclipse-RDV_4.6.csv')
data4=['4.6',draft7,draft8]
draft9 = pd.read_csv('db\\aEclipse-RV_4.7.csv')
draft10 = pd.read_csv('db\\aEclipse-RDV_4.7.csv')
data5=['4.7',draft9,draft10]
allf=[data1,data2,data3,data4,data5]


In [55]:
time=['RV','RDV']
pData=pd.DataFrame()
T=list()
E=list()
results=[None] * 5
parameter=[None] * 5
for i,file in enumerate(allf):
    k=0
    T=list()
    E=list()
    for j,dat in enumerate(file):
        if j==0:
            version=dat
        else:
            T.extend(dat["diff"])
            E.extend( dat["done"])
            k=k+1
           # print(len(T))
        
    results[i]=T
    parameter[i]=E
    #print(i,len(T))
    #print(results[i])
  
   # break
    print(version)
  

4.3
4.4
4.5
4.6
4.7


In [67]:
from scipy.stats import spearmanr, kendalltau, pearsonr
for i in range(4):
    coef, p = spearmanr(results[i], parameter[i])
    print('Spearmans correlation coefficient: %.3f' % coef)
# interpret the significance
    alpha = 0.05
    if p > alpha:
         print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
    else:
         print('Samples are correlated (reject H0) p=%.3f' % p)
   #result = [106, 86, 100, 101, 99, 103, 97, 113, 112, 110]

#parameter = [False, False, True, False, False, False, True, False, True, False]



Spearmans correlation coefficient: 0.005
Samples are uncorrelated (fail to reject H0) p=0.900
Spearmans correlation coefficient: -0.026
Samples are uncorrelated (fail to reject H0) p=0.541
Spearmans correlation coefficient: -0.022
Samples are uncorrelated (fail to reject H0) p=0.751
Spearmans correlation coefficient: -0.007
Samples are uncorrelated (fail to reject H0) p=0.874


In [71]:
from scipy.stats import spearmanr, kendalltau, pearsonr


result = [106, 86, 100, 101, 99, 103, 97, 113, 112, 110]

parameter1 = ['A', 'B', 'A', 'A', 'B', 'A', 'B', 'A', 'A', 'A']

coef, p = spearmanr(result, parameter1)
print('Spearmans correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
    print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
    print('Samples are correlated (reject H0) p=%.3f' % p)

Spearmans correlation coefficient: -0.798
Samples are correlated (reject H0) p=0.006


C:\Users\zeinab\Anaconda3\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [70]:
from numpy.random import rand
from numpy.random import seed
from scipy.stats import spearmanr
# seed random number generator
seed(1)
# prepare data
data1 = rand(1000) * 20
data2 = data1 + (rand(1000) * 10)
# calculate spearman's correlation
coef, p = spearmanr(data1, data2)
print('Spearmans correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

Spearmans correlation coefficient: 0.900
Samples are correlated (reject H0) p=0.000
